# Experiment with Argopy from PR575

<img src="https://raw.githubusercontent.com/euroargodev/argopy/master/docs/_static/argopy_logo_long.png" alt="argopy logo" width="200"/>

If you don't know where to start, have a look to the [PR description (↗)](https://github.com/euroargodev/argopy/pull/575) or:
 - code snippets from the [documentation (↗)](https://argopy.readthedocs.io)
 - or to [tutorial notebooks (↗)](https://argopy.readthedocs.io/en/latest/tutorials.html#tutorials)

Rq: This notebook is intended to be executed from a Binder instance for demonstration purposes

### Set-up and import

In [42]:
import os
import subprocess
if os.getenv('BINDER_SERVICE_HOST'):
    warnings.warn("\nRunning on a Binder instance\nBe aware that your changes won't be saved")
    subprocess.run(["pip", "install", "~/argopy"]) # You first need to install this branch version of Argopy

In [ ]:
# We can now import Argopy
import argopy

## ArgoReferenceTable

Rq: I hesitated to call this class ``ArgoReferenceVocabulary`` or ``ArgoVocabulary``. But since the Argo user's manual uses "Reference Table", I opted to keep this approach and do not stick too much to the NVS jargon (which is still visible in the internal machinery though).

The ``ArgoReferenceTable`` instance holds all the Argo referencing system information:

- the comprehensive logic (use ``SENSOR`` in place of ``R25``),
- the table meta-data are in read-only attributes, e.g. ``art.description`` and ``art.version``, ``art.date``, etc...,
- the list of table values/rows, which are accessible through label-based indexing, e.g. ``art['CTD_TEMP_CNDC']``; this means that the instance can be used almost like a dictionary (e.g. for inclusion assertion and iteration),
- values export/search methods, e.g. ``art.search()``, ``art.to_dataframe()``,  ``art.to_dict()``


### Creation

In [6]:
from argopy.related import ArgoReferenceTable

# Use an Argo parameter name, documented by one of the Argo reference tables:
art = ArgoReferenceTable('SENSOR')

# or a reference table identifier:
art = ArgoReferenceTable('R25')

# or a URN:
art = ArgoReferenceTable.from_urn('SDN:R25::CTD_TEMP')

art

<argo.reference.table> 'R25'/'SENSOR'
long_name: "Argo sensor types"
description: "Terms describing sensor types mounted on Argo floats. Argo netCDF variable SENSOR is populated by R25 altLabel."
uri: http://vocab.nerc.ac.uk/collection/R25/current/
version: 4 (2025-08-29 03:00:01)
keys[33]: 'ACOUSTIC', 'ACOUSTIC_GEOLOCATION', 'BACKSCATTERINGMETER_BBP470', 'BACKSCATTERINGMETER_BBP532', 'BACKSCATTERINGMETER_BBP700', 'BACKSCATTERINGMETER_TURBIDITY', 'CTD_CNDC', 'CTD_PRES', 'CTD_TEMP', 'CTD_TEMP_CNDC' and more ...

### Attributes


In [2]:
# All possible attributes are listed in:
art.attrs

('identifier',
 'parameter',
 'long_name',
 'description',
 'version',
 'date',
 'uri',
 'nvs')

In [3]:
# Reference Table attributes:
art.parameter   # Name of the netcdf dataset parameter filled with values from this table
art.identifier  # Reference Table ID
art.description # [nvs['@graph']['@type']=='skos:Collection']["dc:description"]
art.uri         # [nvs['@graph']['@type']=='skos:Collection']["@id"]
art.version     # [nvs['@graph']['@type']=='skos:Collection']['owl:versionInfo']
art.date        # [nvs['@graph']['@type']=='skos:Collection']['dc:date']

Timestamp('2025-08-29 03:00:01')

In [4]:
# Raw NVS json data:
art.nvs.keys()

dict_keys(['@graph', '@context'])

### Indexing and values

In [7]:
art = ArgoReferenceTable('SENSOR')
art

<argo.reference.table> 'R25'/'SENSOR'
long_name: "Argo sensor types"
description: "Terms describing sensor types mounted on Argo floats. Argo netCDF variable SENSOR is populated by R25 altLabel."
uri: http://vocab.nerc.ac.uk/collection/R25/current/
version: 4 (2025-08-29 03:00:01)
keys[33]: 'ACOUSTIC', 'ACOUSTIC_GEOLOCATION', 'BACKSCATTERINGMETER_BBP470', 'BACKSCATTERINGMETER_BBP532', 'BACKSCATTERINGMETER_BBP700', 'BACKSCATTERINGMETER_TURBIDITY', 'CTD_CNDC', 'CTD_PRES', 'CTD_TEMP', 'CTD_TEMP_CNDC' and more ...

In [9]:
# Values (or concept) within this reference table:
art.keys()   # List of reference values name
art.values() # List of :class:`ArgoReferenceValue`
len(art)     # Number of reference values

33

In [10]:
# Check for values:
'CTD_TEMP_CNDC' in art  # Return True

True

In [12]:
# Index by value key, like a simple dictionary:
art['CTD_TEMP_CNDC']  # Return a :class:`ArgoReferenceValue` instance

<argo.reference.table.value> 'CTD_TEMP_CNDC'
long_name: "Temperature sensor for conductivity on a CTD"
definition: "Specific temperature sensor used with conductivity to calculate salinity on a CTD"
urn: "SDN:R25::CTD_TEMP_CNDC"
uri: http://vocab.nerc.ac.uk/collection/R25/current/CTD_TEMP_CNDC/
version: 1 (2023-11-24 09:35:13)
deprecated: False
reference/parameter: R25/SENSOR
relations[6]:
  - "related" to 6 values : 'R27/RBR_ARGO3', 'R27/RBR_ARGO3_DEEP4K', 'R27/RBR_ARGO', 'R27/RBR_ARGO3_DEEP6K', 'R27/RBR' and 'R27/RBR_CTD'
context: (not loaded yet, use key indexing to load)

In [15]:
# Allows to iterate over all values/concepts:
for concept in art:
    print(f"{concept.name:30s}= {concept.urn}")

ACOUSTIC                      = SDN:R25::ACOUSTIC
ACOUSTIC_GEOLOCATION          = SDN:R25::ACOUSTIC_GEOLOCATION
BACKSCATTERINGMETER_BBP470    = SDN:R25::BACKSCATTERINGMETER_BBP470
BACKSCATTERINGMETER_BBP532    = SDN:R25::BACKSCATTERINGMETER_BBP532
BACKSCATTERINGMETER_BBP700    = SDN:R25::BACKSCATTERINGMETER_BBP700
BACKSCATTERINGMETER_TURBIDITY = SDN:R25::BACKSCATTERINGMETER_TURBIDITY
CTD_CNDC                      = SDN:R25::CTD_CNDC
CTD_PRES                      = SDN:R25::CTD_PRES
CTD_TEMP                      = SDN:R25::CTD_TEMP
CTD_TEMP_CNDC                 = SDN:R25::CTD_TEMP_CNDC
EM                            = SDN:R25::EM
FLOATCLOCK_MTIME              = SDN:R25::FLOATCLOCK_MTIME
FLUOROMETER_CDOM              = SDN:R25::FLUOROMETER_CDOM
FLUOROMETER_CHLA              = SDN:R25::FLUOROMETER_CHLA
IDO_DOXY                      = SDN:R25::IDO_DOXY
OPTODE_DOXY                   = SDN:R25::OPTODE_DOXY
RADIOMETER_DOWN_IRR           = SDN:R25::RADIOMETER_DOWN_IRR
RADIOMETER_DOWN_IRR380    

### Export methods

In [16]:
art = ArgoReferenceTable('SENSOR')
art

<argo.reference.table> 'R25'/'SENSOR'
long_name: "Argo sensor types"
description: "Terms describing sensor types mounted on Argo floats. Argo netCDF variable SENSOR is populated by R25 altLabel."
uri: http://vocab.nerc.ac.uk/collection/R25/current/
version: 4 (2025-08-29 03:00:01)
keys[33]: 'ACOUSTIC', 'ACOUSTIC_GEOLOCATION', 'BACKSCATTERINGMETER_BBP470', 'BACKSCATTERINGMETER_BBP532', 'BACKSCATTERINGMETER_BBP700', 'BACKSCATTERINGMETER_TURBIDITY', 'CTD_CNDC', 'CTD_PRES', 'CTD_TEMP', 'CTD_TEMP_CNDC' and more ...

In [24]:
# Export table attributes to a dictionary (rq: this is not to export values):
art.to_dict()

{'identifier': 'R25',
 'parameter': 'SENSOR',
 'long_name': 'Argo sensor types',
 'description': 'Terms describing sensor types mounted on Argo floats. Argo netCDF variable SENSOR is populated by R25 altLabel.',
 'version': '4',
 'date': Timestamp('2025-08-29 03:00:01'),
 'uri': 'http://vocab.nerc.ac.uk/collection/R25/current/'}

In [25]:
art.to_dict(keys=['parameter', 'date', 'uri'])  # Select Table attributes to export in dictionary keys

{'parameter': 'SENSOR',
 'date': Timestamp('2025-08-29 03:00:01'),
 'uri': 'http://vocab.nerc.ac.uk/collection/R25/current/'}

In [18]:
# Export table values to a pd.DataFrame:
art.to_dataframe()

,name,reference,long_name,definition,deprecated,version,date,uri,urn,parameter,related,broader,narrower,sameas,context
0,ACOUSTIC,R25,Acoustic sensor,Acoustic sensor,False,1,2019-10-11 15:06:36,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::ACOUSTIC,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
1,ACOUSTIC_GEOLOCATION,R25,Acoustic sensor measuring geolocation,Acoustic sensor for geolocation,False,1,2019-10-11 15:06:36,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::ACOUSTIC_GEOLOCATION,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
2,BACKSCATTERINGMETER_BBP470,R25,Backscattering meter measuring backscattering ...,Backscattering meter measuring backscattering ...,False,1,2021-02-09 15:34:44,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::BACKSCATTERINGMETER_BBP470,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
3,BACKSCATTERINGMETER_BBP532,R25,Backscattering meter measuring backscattering ...,Backscattering meter measuring backscattering ...,False,1,2021-02-09 15:34:44,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::BACKSCATTERINGMETER_BBP532,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
4,BACKSCATTERINGMETER_BBP700,R25,Backscattering meter measuring backscattering ...,Backscattering meter measuring backscattering ...,False,1,2021-02-09 15:34:44,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::BACKSCATTERINGMETER_BBP700,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
5,BACKSCATTERINGMETER_TURBIDITY,R25,Backscattering meter measuring turbidity,Backscattering meter measuring turbidity,False,1,2019-10-11 15:06:36,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::BACKSCATTERINGMETER_TURBIDITY,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
6,CTD_CNDC,R25,Conductivity Temperature Depth (CTD) sensors p...,Conductivity Temperature Depth (CTD) sensors p...,False,1,2019-10-11 15:06:36,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::CTD_CNDC,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
7,CTD_PRES,R25,Conductivity Temperature Depth (CTD) sensors p...,Conductivity Temperature Depth (CTD) sensors p...,False,1,2019-10-11 15:06:36,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::CTD_PRES,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
8,CTD_TEMP,R25,Conductivity Temperature Depth (CTD) sensors p...,Conductivity Temperature Depth (CTD) sensors p...,False,1,2019-10-11 15:06:36,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::CTD_TEMP,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None
9,CTD_TEMP_CNDC,R25,Temperature sensor for conductivity on a CTD,Specific temperature sensor used with conducti...,False,1,2023-11-24 09:35:13,http://vocab.nerc.ac.uk/collection/R25/current...,SDN:R25::CTD_TEMP_CNDC,SENSOR,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None,None


In [19]:
art.to_dataframe(columns=['name', 'deprecated'])  # Select value attributes to export in columns

,name,deprecated
0,ACOUSTIC,False
1,ACOUSTIC_GEOLOCATION,False
2,BACKSCATTERINGMETER_BBP470,False
3,BACKSCATTERINGMETER_BBP532,False
4,BACKSCATTERINGMETER_BBP700,False
5,BACKSCATTERINGMETER_TURBIDITY,False
6,CTD_CNDC,False
7,CTD_PRES,False
8,CTD_TEMP,False
9,CTD_TEMP_CNDC,False


In [41]:
# Export table values to a dictionary, use pd.DataFrame:
art.to_dataframe(columns=['name', 'deprecated']).to_dict(orient='records')

[{'name': 'AANDERAA_OPTODE', 'deprecated': False},
 {'name': 'AANDERAA_OPTODE_3830', 'deprecated': False},
 {'name': 'AANDERAA_OPTODE_3835', 'deprecated': False},
 {'name': 'AANDERAA_OPTODE_3930', 'deprecated': False},
 {'name': 'AANDERAA_OPTODE_4330', 'deprecated': False},
 {'name': 'AANDERAA_OPTODE_4330F', 'deprecated': False},
 {'name': 'AANDERAA_OPTODE_4831', 'deprecated': False},
 {'name': 'AANDERAA_OPTODE_4831F', 'deprecated': False},
 {'name': 'AMETEK', 'deprecated': False},
 {'name': 'AMETEK_3000PSIA', 'deprecated': False},
 {'name': 'AROD_FT', 'deprecated': False},
 {'name': 'ARO_FT', 'deprecated': False},
 {'name': 'CTD_F01', 'deprecated': False},
 {'name': 'CYCLOPS_7_FLUOROMETER', 'deprecated': False},
 {'name': 'C_ROVER', 'deprecated': False},
 {'name': 'DRUCK', 'deprecated': False},
 {'name': 'DRUCK_10153PSIA', 'deprecated': False},
 {'name': 'DRUCK_2900PSIA', 'deprecated': False},
 {'name': 'DSB301-10-C85', 'deprecated': False},
 {'name': 'DURA', 'deprecated': False},
 {'

### Search method

In [30]:
art = ArgoReferenceTable('SENSOR_MODEL')
art

<argo.reference.table> 'R27'/'SENSOR_MODEL'
long_name: "Argo sensor models"
description: "Terms listing models of sensors mounted on Argo floats. Note: avoid using the manufacturer name and sensor firmware version in new entries when possible. Argo netCDF variable SENSOR_MODEL is populated by R27 altLabel."
uri: http://vocab.nerc.ac.uk/collection/R27/current/
version: 23 (2025-11-19 02:00:00)
keys[137]: 'AANDERAA_OPTODE', 'AANDERAA_OPTODE_3830', 'AANDERAA_OPTODE_3835', 'AANDERAA_OPTODE_3930', 'AANDERAA_OPTODE_4330', 'AANDERAA_OPTODE_4330F', 'AANDERAA_OPTODE_4831', 'AANDERAA_OPTODE_4831F', 'AMETEK', 'AMETEK_3000PSIA' and more ...

In [33]:
# Search methods (return a list of :class:`ArgoReferenceValue` with match):
# Any of the :class:`ArgoReferenceValue` attribute can be searched
art.search(name='RAMSES')         # Search in values name

[<argo.reference.table.value> 'RAMSES_ACC'
 long_name: "TriOS RAMSES ACC spectral imaging radiometer"
 definition: "A spectral imaging radiometer with cosine detector which measures radiance, irradiance or scalar irradiance in the Ultraviolet (UV), Vis (Visible) and Ultraviolet-visible (UV-Vis) ranges, manufactured by TriOS. Number of channels: 256."
 urn: "SDN:R27::RAMSES_ACC"
 uri: http://vocab.nerc.ac.uk/collection/R27/current/RAMSES_ACC/
 version: 1 (2021-07-01 16:48:21)
 deprecated: False
 reference/parameter: R27/SENSOR_MODEL
 relations[3]:
   - 1 "broader" value: 'R26/TRIOS'
   - "related" to 2 values : 'R25/RADIOMETER_PAR' and 'R25/RADIOMETER_DOWN_IRR'
 context: (not loaded yet, use key indexing to load)]

In [34]:
art.search(definition='imaging')  # Search in values definition

[<argo.reference.table.value> 'RAMSES_ACC'
 long_name: "TriOS RAMSES ACC spectral imaging radiometer"
 definition: "A spectral imaging radiometer with cosine detector which measures radiance, irradiance or scalar irradiance in the Ultraviolet (UV), Vis (Visible) and Ultraviolet-visible (UV-Vis) ranges, manufactured by TriOS. Number of channels: 256."
 urn: "SDN:R27::RAMSES_ACC"
 uri: http://vocab.nerc.ac.uk/collection/R27/current/RAMSES_ACC/
 version: 1 (2021-07-01 16:48:21)
 deprecated: False
 reference/parameter: R27/SENSOR_MODEL
 relations[3]:
   - 1 "broader" value: 'R26/TRIOS'
   - "related" to 2 values : 'R25/RADIOMETER_PAR' and 'R25/RADIOMETER_DOWN_IRR'
 context: (not loaded yet, use key indexing to load)]

In [35]:
art.search(long_name='TriOS')     # Search in values long name

[<argo.reference.table.value> 'OPUS_DS'
 long_name: "TriOS OPUS spectral sensor - Deep Sea version"
 definition: "A spectral sensor designed to measure nitrogen and carbon compounds, with titanium housing and depth-rating of 6000 dbar (Deep-Sea version). A xenon flash lamp provides the light source, and a 256-channel spectrometer covers a wavelength range of 200 nm - 360 nm. The instrument is manufactured by TriOS."
 urn: "SDN:R27::OPUS_DS"
 uri: http://vocab.nerc.ac.uk/collection/R27/current/OPUS_DS/
 version: 1 (2021-08-18 14:32:12)
 deprecated: False
 reference/parameter: R27/SENSOR_MODEL
 relations[3]:
   - 1 "broader" value: 'R26/TRIOS'
   - "related" to 2 values : 'R25/SPECTROPHOTOMETER_NITRATE' and 'R25/SPECTROPHOTOMETER_BISULFIDE'
 context: (not loaded yet, use key indexing to load),
 <argo.reference.table.value> 'RAMSES_ACC'
 long_name: "TriOS RAMSES ACC spectral imaging radiometer"
 definition: "A spectral imaging radiometer with cosine detector which measures radiance, irrad

In [37]:
# Possible change to output format:
art.search(long_name='TriOS', output='df')  # To a :class:`pd.DataFrame`

,name,reference,long_name,definition,deprecated,version,date,uri,urn,parameter,related,broader,narrower,sameas,context
0,OPUS_DS,R27,TriOS OPUS spectral sensor - Deep Sea version,A spectral sensor designed to measure nitrogen...,False,1,2021-08-18 14:32:12,http://vocab.nerc.ac.uk/collection/R27/current...,SDN:R27::OPUS_DS,SENSOR_MODEL,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None
1,RAMSES_ACC,R27,TriOS RAMSES ACC spectral imaging radiometer,A spectral imaging radiometer with cosine dete...,False,1,2021-07-01 16:48:21,http://vocab.nerc.ac.uk/collection/R27/current...,SDN:R27::RAMSES_ACC,SENSOR_MODEL,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,[{'@id': 'http://vocab.nerc.ac.uk/collection/R...,None,None,None
